<a href="https://colab.research.google.com/github/pacoshu/HMC/blob/main/Copy_of_HMC_TEST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import scipy.ndimage
from mpl_toolkits.axes_grid1 import make_axes_locatable
from random import randint
import itertools
import matplotlib as mpl


#HMC parameters
higherorder = False #True: 4th order; false: 2nd order leapfrog

numplot=10  # write outputs each numplot iterations
diagprint=True # print out more information

#numerical parameters
nc = 64 # Number of cells in one dimension, i.e., total number is nc^3
Ngibbs = 300



#data parameters
L = 1250 # Box length [Mpc/h]
bias = 1.8 # bias if equal to 1 it is unbiased
NmeanG = 3e-4 * L**3/np.float64(nc**3) # mean number density used in the inverseCrime case
dgrowth2=0.74*0.74  # physical parameter growth factor, normalisation of the Pk

seed = 1
Nb = 100  # Number of bins

#HMC parameters
stepsize = 0.1 # 6e-2 # basic stepsize
Nsteps = 5 #  relevant for 2nd order
nrep = 3 # relevant for 4th order: nrep x forward steps + 1 x backward step + nrep x forward steps

inverseCrime=True # if False read Nobs from an N-body simulation, if True generate Poisson-Lognormal sample data

maxvalue=10000 # safety upper signal value limit




In [ ]:

# ****************************************************************************************

""" these fft definitions permit us to change the normalization with nct, if wished """
def fft(inarr,nct):
    finarr= np.fft.fftn(inarr)

    return (finarr)

def ifft(finarr,nct):
    inarr= np.fft.ifftn(finarr)

    return (inarr)
  
"""Function to obtain kx, ky and kz in the box (nc x nc x nc)"""
def k_squared(L,nc,i,j,k):
    
      kfac = 2.0*np.pi/L

      if i <= nc/2:
        kx = kfac*np.float64(i)
      else:
        kx = -kfac*np.float64(nc-i)
      if j <= nc/2:
        ky = kfac*np.float64(j)
      else:
        ky = -kfac*np.float64(nc-j)
      if k <= nc/2:
        kz = kfac*k
      else:
        kz = -kfac*np.float64(nc-k)
      k2 = kx**2+ky**2+kz**2

      return float(k2)

""" Funtion to calculate the power spectrum in spherical bins """
def measure_spectrum(signal, kmode, power, nc, L, N_bin):
    
      fsignal = fft(signal,nc**3) #np.fft.fftn(signal)

      kmax = np.sqrt(k_squared(L,nc,nc/2,nc/2,nc/2))
      dk = kmax/np.float64(N_bin)  # Bin width
      nmode = np.zeros((N_bin))
      for i in range(nc):
        for j in range(nc):
            for k in range(nc):
                ktot = np.sqrt(k_squared(L,nc,i,j,k))
                nbin = int(ktot/dk-0.5)
                akl = fsignal.real[i,j,k]
                bkl = fsignal.imag[i,j,k]
                kmode[nbin]+=ktot
                power[nbin]+=(akl*akl+bkl*bkl)
                nmode[nbin]+=1
      for m in range(N_bin):
        if(nmode[m]>0):
            kmode[m]/=nmode[m]
            power[m]/=nmode[m]


      return (kmode, power)

# ****************************************************************************************

def pk1Dto3D(filename, nc, L, N_bin):

    # Theoretical Power Spectrum
    power = np.loadtxt(filename)
    kmodet = power[:,0]*1.00
    pkt = power[:,1]*dgrowth2

    ktot = np.zeros((nc,nc,nc))

    for i in range(nc):
      for j in range(nc):
        for k in range(int(nc)):
          ktot[i,j,k] = np.sqrt(k_squared(L,nc,i,j,k))
  
    # Interpolation to obtain the power spectrum from the theoretical values
    pk = np.interp(ktot, kmodet, pkt)*nc**6/L**3

    return pk

def plotdiagn(inarr, mu, nc, L, N_bins, vmin, vmax):
  
    if mu!=0:
      indelta=np.exp(inarr+mu)-1.
    else:
      indelta=inarr


    power = np.loadtxt(filename)
    kmodet = power[:,0]
    pkt = power[:,1]*dgrowth2

    kmodes = np.zeros(N_bins)
    powers = np.zeros(N_bins)

    measure_spectrum(indelta, kmodes, powers, nc, L, N_bins)

    plt.ion()
    plt.clf()
    plt.semilogx() 
    plt.semilogy()

    plt.xlim(1e-3,2e0)

    if higherorder==False: 
        plt.ylim(1e2, 5e10)      
    else:   
        plt.ylim(1e2, 5e10)
        
    

    plt.plot(kmodet,pkt*(nc**6/L**3), 'r')
    plt.plot(kmodes, powers, 'k')

    plt.xlabel(r'$k[h\,\mathrm{Mpc}^{-1}]$',fontsize=25)
    plt.ylabel(r'$P(k)$',fontsize=20, labelpad=10)
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
   
    plt.show()

    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(111)

    start=0
    end=1
    #c=indelta[:,:,start:end].mean(axis=-1)

    c=inarr[:,:,0]

    c = scipy.ndimage.interpolation.zoom(c ,order=3, zoom=1)

    def truncate_colormap(cmap, minval=-4.0, maxval=1.0, n=100):
        new_cmap = colors.LinearSegmentedColormap.from_list(
          'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
          cmap(np.linspace(minval, maxval, n)))
        return new_cmap

    cmap = plt.get_cmap('bone_r')
    cmap = plt.get_cmap('nipy_spectral')
    cmap = plt.get_cmap('jet')
    cmap.set_under('w')


    im = plt.imshow(c,interpolation='bilinear',aspect='auto', extent=[0,L,0,L],cmap=cmap) #, vmin = vmin, vmax = vmax)
    plt.xlabel(r'$y[h^{-1}\,\mathrm{Mpc}]$',fontsize=25)
    plt.ylabel(r'$x[h^{-1}\,\mathrm{Mpc}]$',fontsize=25, labelpad=10)

    ax.set_aspect('equal')
    plt.setp(ax.get_xticklabels(), fontsize=18)
    plt.setp(ax.get_yticklabels(), fontsize=18)

    divider = make_axes_locatable(plt.gca())
    cax = divider.append_axes("top", "5%", pad="1%")
    cb = plt.colorbar(im,orientation='horizontal',cax=cax)
    cb.ax.get_xaxis().labelpad = 0.
    cb.ax.xaxis.set_tick_params(labelsize=18)
    cb.ax.xaxis.set_ticks_position('top')

    plt.show()


def create_GaussianField(pk, seed, nc, norm):

# Gaussian field

    mu = 0.0
    sigma = 1.0

    gdis1D = np.random.normal(mu, sigma, nc**3)
    gdis = gdis1D.reshape(nc,nc,nc)

    # Gaussian field in fourier space
    fgdist = fft(gdis,nc**3)

    # We calculate the overdensity field by multiplying the gaussian field by the sqrt of the interpolated Pk

    fgdist.real*=np.sqrt(pk*norm)
    fgdist.imag*=np.sqrt(pk*norm)

    xg = ifft(fgdist,nc**3)
    xgr = xg.real

    return xgr

def calc_conviKw(inarr, kernel, normiK):
    """ Convolution of an input array and the inverse of a kernel"""
    finarr = fft(inarr,nc**3) #np.fft.fftn(inarr)

    kernelp=kernel
    kernelp[0,0,0]=1.0
    invkernel = 1./np.float64(kernelp) # S^-1
    invkernel[0,0,0]=0.0
    
    fconviKwIN = invkernel*normiK*finarr
    conviKwIN = ifft(fconviKwIN,nc**3) #np.fft.ifftn(fconviKwIN)
    conviKwINr = conviKwIN.real

    return conviKwINr

"""Definitions to obtain lambda: the expectation value of galaxy counts for each cell"""
def calc_mu(signal, nc):
    #mu=<ln(1+delta)>   

      flag=False

      abssignal=(signal)

      if any(abssignal[abssignal> maxvalue])==True:      
        flag = True
        signal*=0.

      return (float(-np.log(np.mean(np.float64(np.exp(signal))))))


def calc_fmean(signal, NmeanG, nc, mu, bias):

      flag=False

      abssignal=(signal+mu)

      if any(abssignal[abssignal> maxvalue])==True:      
        flag = True
        signal*=0.

      return (flag, float(NmeanG/(np.mean(np.float64(np.exp((signal+mu)*bias))))))

"""Function to calculate the minus logarithm of the likelihood"""
def calc_nloglikel(signal, Nobs, nc, fmean, mu):

      flag=False      

      abssignal=(signal)

      if any(abssignal[abssignal> maxvalue])==True:      
        flag = True
        signal*=0.

      return (flag, float(np.sum(np.float64(np.float64(fmean)*np.float64(np.exp((signal+mu)*bias))-np.float64(Nobs)*np.float64(mu+signal)*np.float64(bias)))))
        
            
"""Function to calculate the minus logarithm of the prior"""
def calc_nlogprior(signal, pk, nc, normiK):
      convipkws=calc_conviKw(signal, pk, normiK)

      flag=False

      abssignal=(signal)

      if any(abssignal[abssignal> maxvalue])==True:      
        flag = True
        signal*=0.

      return (float(np.sum(np.float64(np.float64(0.5)*np.float64(signal)*np.float64(convipkws)))))

"""Function to calculate the kinetic energy"""
def calc_kineticE(momenta, mass, nc, normiK):

    conviMasswMom=calc_conviKw(momenta, mass, normiM)

    return (float(np.sum(np.float64(np.float64(0.5)*np.float64(momenta)*np.float64(conviMasswMom)))))



"""Function to calculate the Hamiltonian"""
def calc_HamiltonianE(signal, momenta, nc, pk, mass, Nobs, fmean, mu, normiK, normiM):
    kineticE = calc_kineticE(momenta, mass, nc, normiM)
    nloglikel=np.float64(0.)
    (flag, nloglikel)=calc_nloglikel(signal, Nobs, nc, fmean, mu)
    nlogprior=calc_nlogprior(signal, pk, nc, normiK)
    potentialE = np.float64(nlogprior)+np.float64(nloglikel)
    if diagprint==True:
      print("logk=%f"%(kineticE),"logp=%f"%(nlogprior),"loglike=%f"%(nloglikel))
    HamiltonianE = np.float64(kineticE)+np.float64(potentialE)
    
    return (flag,float(HamiltonianE))

#Function to calculate the partial derivative of the potential energy respect to the signal
def calc_grad_E(signal, Nobs, nc, pk, fmean, mu, bias, normiK):

      convipkws=calc_conviKw(signal, pk, normiK)
   
      flag=False

      abssignal=(signal+mu)

      if any(abssignal[abssignal> maxvalue])==True:      
        flag = True
        signal*=0.

      grad_E=np.float64(convipkws)+np.float64(bias*(fmean*np.exp((signal+mu)*bias)-np.float64(Nobs)))
      
      return (flag,grad_E)
     

"""Leapfrog algorithm"""
#1) pi(t+e/2)=pi(t)-e/2*gradPsi(si(t))
#2) si(t+e)=si(t)+e/Mi*pi(t+e/2)
#3) pi(t+e)=pi(t+e/2)-e/2*gradPsi(si(t+e))

def leapfrog(Nobs, signal, momenta, pk, mass, nc, fmean, mu, bias, normiK, normiM, epsilon, grad_E):

    #this line can be saved
    #(flag,grad_E)=calc_grad_E(signal, Nobs, nc, pk, fmean, mu, bias, normiK) 
    
    momental=np.float64(momenta)-np.float64(0.5*epsilon*grad_E)

    conviMasswMom=calc_conviKw(momental, mass, normiM)

    signalf=np.float64(signal)+np.float64(epsilon*conviMasswMom)
    (flag,grad_Ef)=calc_grad_E(signalf, Nobs, nc, pk, fmean, mu, bias, normiK)
    momentaf=np.float64(momental)-np.float64(0.5*epsilon*grad_Ef)
      
    return (flag, signalf, momentaf, grad_Ef)


def Hamiltonian_sampling(higherorder, signal, Nobs, pk, nc, L, fmean, mu, bias, seed, normiK, normiM, normMom, stepsize, Nsteps, nrep):   

    pkp=pk   
    pkp[pkp==0.]=-1.

    mass=1./pkp
    mass[mass<0.]=0.


    signalinit=signal

    signalf=signal
       
    acceptance=False
    ncount=0

    epsarr=False  

    (flag,grad_E0)=calc_grad_E(signalinit, Nobs, nc, pk, fmean, mu, bias, normiK)

    itcount=1

    while acceptance==False:

        grad_E=grad_E0

        flag=False
        
        momentainit=create_GaussianField(mass, seed, nc, normMom) 
	    
        momentaf=momentainit
         
        (signalf, momentaf)=(signalinit, momentainit) 
        (signal1, momenta1)=(signalinit, momentainit)

        if higherorder==False: 

                if diagprint==True:
                  print("2nd order integrator")

                Nstepsf = int(Nsteps*random.uniform(0,1)+0.5)
                itcount+=Nstepsf

                epsilon = stepsize*random.uniform(0,1)

                for m in range(Nstepsf):
                    (flag, signal1, momenta1, grad_E)=leapfrog(Nobs, signal1, momenta1, pk, mass, nc, fmean, mu, bias, normiK, normiM, epsilon, grad_E)                                    
   
        else: 

                if diagprint==True:
                  print("4th order integrator")


                epsilon = stepsize*random.uniform(0,1)

                epsilonback = -np.float64(epsilon)*((np.float64(2.)*np.float64(nrep))**(np.float64(1./3.)))              


                Nstepsf=0

                for m in range(nrep):

                    Nstepsf+=1

                    (flag, signal2, momenta2, grad_Ef)=leapfrog(Nobs, signal1, momenta1, pk, mass, nc, fmean, mu, bias, normiK, normiM, epsilon, grad_E)                  
                    grad_E=grad_Ef
                    (signal1,momenta1)=(signal2,momenta2)

                (flag, signal2, momenta2, grad_Ef)=leapfrog(Nobs, signal1, momenta1, pk, mass, nc, fmean, mu, bias, normiK, normiM, epsilonback, grad_E)                             
                grad_E=grad_Ef
                (signal1,momenta1)=(signal2,momenta2)
                Nstepsf+=1

                for m in range(nrep):

                    Nstepsf+=1
         
                    (flag, signal2, momenta2, grad_Ef)=leapfrog(Nobs, signal1, momenta1, pk, mass, nc, fmean, mu, bias, normiK, normiM, epsilon, grad_E) 
                    grad_E=grad_Ef
                    (signal1,momenta1)=(signal2,momenta2)

                itcount+=Nstepsf
 	        
        if diagprint==True:
          print("HMC gradient evaluations=%i"%(Nstepsf))
          

        (signalf, momentaf)=(signal1, momenta1)            
        
        Hf=0.
        Hi=0.
        (flag, Hf)=calc_HamiltonianE(signalf, momentaf, nc, pk, mass, Nobs, fmean, mu, normiK, normiM)
        (flag, Hi)=calc_HamiltonianE(signalinit, momentainit, nc, pk, mass, Nobs, fmean, mu, normiK, normiM)
        if diagprint==True:
          print("Hf=%f"%(Hf),"Hi=%f"%(Hi))
        dH=(np.float64(Hf)-np.float64(Hi))
        if diagprint==True:
          print("dH=%f"%(dH))
        
        pacc=1.
        if -dH<0.:
            if np.exp(-dH)<1.:
                pacc=np.exp(-dH)
                        
        uacc=1
        if pacc>=1.0:
            acceptance=True
        else:
            uacc=random.uniform(0,1)
        if uacc<=pacc:
            acceptance=True
            
        if flag==True:
            acceptance=False
            if diagprint==True:
              print("flag=True")
        if diagprint==True:     
         print("uacc=",uacc,"pacc=",pacc)
        
        if acceptance==True:
          if diagprint==True:
            print("accepted!")
        
        ncount+=1

    if diagprint==True:
      print("------------------------------->")
      print("HMC iterations=%i"%(ncount))
      print("------------------------------->")

    return (signalf, itcount)

def Gibbs_sampling(higherorder,signal, Nobs, pk, Ngibbs, nc, L, NmeanG, fmean, mu, bias, N_bins, filename, seed, normiK, normiM, normMom, stepsize, Nsteps, nrep, vmin, vmax):

    signalf=signal
    signall=signal
 
    ittot=0

    for l in range(Ngibbs):

        if 0==l%numplot:
          print("------------------------------->")
          print("---->iteration=%i"%(l+1))

        #automatic sampling of mu, works on large meshes
        mu=calc_mu(signalf, nc)

        if 0==l%numplot:
          print("mu=%f"%(mu))

        #fmean=NmeanG
        flag=False
        #automatic sampling of fmean, relevant for nonlinear bias
        (flag, fmean)=calc_fmean(signalf, NmeanG, nc, mu, bias)

        if 0==l%numplot:
          print("fmean=%f"%(fmean))

        (signalf, itcount)=Hamiltonian_sampling(higherorder,signall, Nobs, pk, nc, L, fmean, mu, bias, seed, normiK, normiM, normMom, stepsize, Nsteps, nrep)
	
        signall=signalf

        ittot+=itcount

        print("total HMC gradient evaluations=%i"%(ittot))
	
        if 0==l%numplot:
            deltadm=np.exp(signalf+mu)-1.
            plotdiagn(signalf, 0, nc, L, N_bins, vmin, vmax)

    return (signalf)

def calc_pklog(pk, nc):
  
    pk/=np.float64(nc**3)
    fcorr=ifft(pk,nc**3)
    corr=fcorr.real
    corrlog=np.log(np.float64(1.)+np.float64(corr))
    pklog=(np.abs(fft(corrlog,nc**3))*np.float64(nc**3))
 
    return (pklog.real)


#sample_lognormal_poisson_signal():

#plotting
vmin=-1.8
vmax=1.8

#Fourier definition dependent normalisations
normiK=1.*np.float64(nc**3)
normiM=1./np.float64(nc**3)
normMom=1.*np.float64(nc**3)
normDelta=1./np.float64(nc**3)

filename="/content/drive/My Drive/Pk.input_zinit_normalized_at_z0.dat"

try:

  if higherorder==False: 
    print("2nd order integrator")
  else:
    print("4th order integrator")

  np.random.seed(seed) 
   
  pklin=pk1Dto3D(filename, nc, L, Nb)

  pk=pklin

  #lognormal prior Pk, works on large meshes
  #if inverseCrime==True:
  #  pk=calc_pklog(pklin, nc)
  
  pk[0,0,0]=0.

  deltadmG=create_GaussianField(pk, seed, nc, normDelta)
  muG=calc_mu(deltadmG, nc)

  print("muG=%f"%(muG))

  deltadm=np.exp(deltadmG+muG)-1.
   
  if inverseCrime==True:

    print("------------------------------->")
    print("Input signal we aim to infer--->")
    plotdiagn(deltadmG, 0, nc, L, Nb, vmin, vmax)

    fmean=NmeanG
    (flag, fmean)=calc_fmean(deltadmG, NmeanG, nc, muG, bias)

    print("NmeanG=%f"%(NmeanG),"fmean=%f"%(fmean))

    #rhoG=fmean*(1.+deltadm)**bias

    rhoG=fmean*np.exp((deltadmG+muG)*bias)

    Nobs=np.random.poisson(rhoG)
    Nobs=np.asarray(Nobs, dtype = np.float32, order ='C')

  else:

    if nc==64:
      file="/content/drive/My Drive/nobs64.dat"
    if nc==128:
      file="/content/drive/My Drive/nobs128.dat"

    datafile = np.fromfile(file,dtype=np.float32)
    Nobs= datafile.reshape((nc,nc,nc))

    NmeanG=np.mean(Nobs)

    (flag, fmean)=calc_fmean(deltadmG, NmeanG, nc, muG, bias)

    print("NmeanG=%f"%(NmeanG),"fmean=%f"%(fmean))

  deltaobs=Nobs/np.mean(Nobs)-1

  vminn=np.min(deltaobs)
  vmaxn=np.max(deltaobs)

  print("------------------------------->")
  print("Input data--->")
  plotdiagn(deltaobs, 0, nc, L, Nb, vminn, vmaxn)

  signal=np.zeros((nc,nc,nc))
  signalf=signal

  print("------------------------------->")
  print("Bayesian inference--->")
  signalf=Gibbs_sampling(higherorder,signal, Nobs, pk, Ngibbs, nc, L, NmeanG, fmean, muG, bias, Nb, filename, seed, normiK, normiM, normMom, stepsize, Nsteps, nrep, vmin, vmax)

  print("------------------------------->")
  print("Plot data again for comparison--->")
  plotdiagn(deltaobs, 0, nc, L, Nb, vminn, vmaxn)

except KeyboardInterrupt:
    pass
